In [1]:
!pip install scipy

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple


In [2]:
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
import numpy as np

In [3]:
class lstm(nn.Module):
    def __init__(self,num):
        super().__init__()
        self.lstm = nn.LSTM(1,20,2,dropout=0.1,batch_first=True)
        self.linear = nn.Linear(20,5)
        self.relu=nn.ReLU()
        self.linear1 = nn.Linear(5,1)
        self.num = num
        self.lstm1 = nn.LSTM(1,20,2,dropout=0.2,batch_first=True)
    def forward(self,x):
        #print(x.shape)
        x = x.unsqueeze(2)
        output, (hn, cn) = self.lstm(x)
        output1=[]
        
        for i in range(self.num):
            output, (hn, cn) = self.lstm1(x[:,-1,:].unsqueeze(1),(hn,cn))
            out = self.relu(self.linear(output[:,-1,:]))
            out1 = self.linear1(out)
            if i <=self.num-1:
                out1 = out1.unsqueeze(2)
                output1.append(out1)
                #print(out1.shape)
                x = torch.cat([x,out1],dim=1)
                output, (hn, cn) = self.lstm(x[:,1:,:])
        return torch.cat(output1,dim=0)

In [4]:
model = lstm(2)
model.to('cpu')

lstm(
  (lstm): LSTM(1, 20, num_layers=2, batch_first=True, dropout=0.1)
  (linear): Linear(in_features=20, out_features=5, bias=True)
  (relu): ReLU()
  (linear1): Linear(in_features=5, out_features=1, bias=True)
  (lstm1): LSTM(1, 20, num_layers=2, batch_first=True, dropout=0.2)
)

In [5]:
loadmodel = torch.load('forward_model.pt',map_location='cpu')
model.load_state_dict(loadmodel)

<All keys matched successfully>

In [6]:
data = pd.read_csv('result_eva4.csv')

In [7]:
data = data[data['name']=='cluster']

In [8]:
data.to_csv('lstmdata.csv')

In [9]:
data_time = data['time']

In [10]:
data_x = data['dcp']

In [11]:
data_x = data_x[:42]

In [12]:
data_x = data_x.to_numpy()

In [13]:
input_x=data_x[-5:]
input_x = torch.from_numpy(input_x)
input_x = input_x.reshape(1,5).float()

In [14]:
output = model(input_x)
output = torch.flatten(output)

In [15]:
output = output.detach().numpy()
output

array([427.81284, 427.81293], dtype=float32)

In [16]:
data_x = np.concatenate([data_x,output])
data_x

array([ 12.        , 234.        , 306.        , 298.        ,
       180.        , 248.        , 268.        , 212.        ,
       626.        , 992.        , 306.        , 270.        ,
       250.        , 294.        , 300.02758621,  82.        ,
        52.        , 306.        , 210.        , 356.19631902,
       334.        , 251.44186047,  66.        , 332.        ,
       326.        , 204.        , 262.        , 244.        ,
       102.        , 312.        , 136.        ,  90.        ,
       386.        ,  82.        , 214.        ,  70.        ,
       127.8125    , 508.        , 206.        , 290.        ,
       264.        , 383.52830189, 427.81283569, 427.81292725])

In [17]:
loadmodel = torch.load('back_model.pt',map_location='cpu')
model.load_state_dict(loadmodel)

<All keys matched successfully>

In [18]:
input_x=data_x[::-1]
input_x=data_x[-5:]
input_x = torch.from_numpy(input_x)

input_x = input_x.reshape(1,5).float()

In [19]:
output = model(input_x)
output = torch.flatten(output)
output = output.detach().numpy()
output = output[::-1]

In [20]:
data_x = np.concatenate([output,data_x])
data_x 

array([  5.17459631,   5.17459631,  12.        , 234.        ,
       306.        , 298.        , 180.        , 248.        ,
       268.        , 212.        , 626.        , 992.        ,
       306.        , 270.        , 250.        , 294.        ,
       300.02758621,  82.        ,  52.        , 306.        ,
       210.        , 356.19631902, 334.        , 251.44186047,
        66.        , 332.        , 326.        , 204.        ,
       262.        , 244.        , 102.        , 312.        ,
       136.        ,  90.        , 386.        ,  82.        ,
       214.        ,  70.        , 127.8125    , 508.        ,
       206.        , 290.        , 264.        , 383.52830189,
       427.81283569, 427.81292725])

In [21]:
def np_move_avg(a,n,mode="same"):
    return(np.convolve(a, np.ones((n,))/n, mode=mode))

In [22]:
def move_avg(a,n,mode="same"):
    return(np.convolve(a, np.ones((n,))/(n+1), mode=mode))

In [23]:
def compute_fj(x,window):
    out = np_move_avg(x,window)
    out1 = x-out
    #length = len(x)//7
    out1 =out1- np.median(out1)
    return out1
    
    

In [24]:
def compute_srbo(x):
    fft_y = scipy.fft.fft(x)
    abs_y=np.abs(fft_y)                # 取复数的绝对值，即复数的模(双边频谱)
    angle_y=np.angle(fft_y) 
    print(abs_y)
    l=np.log2(abs_y+0.001)
    kernel = np.ones(3)
    h = kernel/3.
    print(h)
    hq=np.convolve(l, h, mode='same')
    r = l-hq
    out_put = r+1j*angle_y
    output = np.abs(scipy.fft.ifft(out_put))
    return output

In [25]:
window = 7

In [26]:
out = compute_fj(data_x,window)

In [27]:
output = compute_srbo(out)

[1145.09749399  375.27124254  226.82721872  586.12542462  780.76182635
  542.72019288  978.29940996  852.42742895 2371.69249389 1220.34595901
 1608.51314184 1024.15996079 1165.34432799 1054.79757532  204.25136171
  612.55202864 1289.00813478  238.43389391 1311.98501958  215.96481295
  736.66395716  517.35898147  400.03578822 1172.63503727  400.03578822
  517.35898147  736.66395716  215.96481295 1311.98501958  238.43389391
 1289.00813478  612.55202864  204.25136171 1054.79757532 1165.34432799
 1024.15996079 1608.51314184 1220.34595901 2371.69249389  852.42742895
  978.29940996  542.72019288  780.76182635  586.12542462  226.82721872
  375.27124254]
[0.33333333 0.33333333 0.33333333]


In [28]:
def compute_d(x,window):
    out = move_avg(x,window)
    d = (x-out)/out
    return d

In [29]:
output_d = compute_d(output,window) 
output_d

array([ 9.67205290e-01, -1.57987597e-01,  1.01046645e+00, -6.00496037e-01,
        1.65883374e+00, -8.64490478e-01,  1.00394610e+00, -4.20267367e-01,
       -4.37085255e-01, -1.03563866e-01, -3.01099153e-01,  2.82940844e+00,
       -4.24853859e-01, -6.21156339e-01, -9.68936415e-04,  4.72454520e-01,
        2.05593481e-01, -2.23241464e-01,  6.80009927e-01, -6.61571826e-01,
        9.08615254e-01, -5.80051444e-01,  1.01104721e-01,  9.16489628e-01,
       -6.06753676e-01, -6.84403823e-01, -9.45807362e-02,  1.09948856e+00,
        2.22480429e+00, -5.76646314e-01, -4.55306586e-01, -2.16828722e-01,
       -9.16661408e-01, -2.34833676e-01,  5.15592648e-01,  1.52288260e+00,
       -6.84426759e-01,  8.22038803e-01,  9.98847021e-02, -2.35901974e-01,
       -6.39466608e-01,  3.37554308e-01, -6.96364136e-02,  5.37322264e-01,
        1.72442394e+00,  2.45718476e-01])

In [30]:
output_d = output_d[2:-2]
output_d

array([ 1.01046645e+00, -6.00496037e-01,  1.65883374e+00, -8.64490478e-01,
        1.00394610e+00, -4.20267367e-01, -4.37085255e-01, -1.03563866e-01,
       -3.01099153e-01,  2.82940844e+00, -4.24853859e-01, -6.21156339e-01,
       -9.68936415e-04,  4.72454520e-01,  2.05593481e-01, -2.23241464e-01,
        6.80009927e-01, -6.61571826e-01,  9.08615254e-01, -5.80051444e-01,
        1.01104721e-01,  9.16489628e-01, -6.06753676e-01, -6.84403823e-01,
       -9.45807362e-02,  1.09948856e+00,  2.22480429e+00, -5.76646314e-01,
       -4.55306586e-01, -2.16828722e-01, -9.16661408e-01, -2.34833676e-01,
        5.15592648e-01,  1.52288260e+00, -6.84426759e-01,  8.22038803e-01,
        9.98847021e-02, -2.35901974e-01, -6.39466608e-01,  3.37554308e-01,
       -6.96364136e-02,  5.37322264e-01])

In [31]:
result = (output_d>1).astype(np.int8)

In [32]:
result

array([1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int8)

In [33]:
np.save('backward_result.npy',output)
np.save('backward_res.npy',output)

In [34]:
o = np.argsort(-output_d)
o.shape

(42,)

In [35]:
output[o]

array([0.22644839, 0.27555194, 0.2787966 , 0.14429174, 0.09663316,
       0.15182926, 0.46776831, 0.06057803, 0.24999395, 0.5901327 ,
       0.17709552, 0.30331484, 0.01617418, 0.09265162, 0.14403939,
       0.2416556 , 0.28901041, 0.07886025, 0.14420833, 0.07825835,
       0.07578282, 0.10555395, 0.1228782 , 0.23432161, 0.18903028,
       0.44027937, 0.13375133, 0.02417101, 0.15456148, 0.34433359,
       0.96843096, 0.60077067, 0.04627354, 0.11422001, 0.13772726,
       0.91201988, 0.26060215, 0.13316728, 0.29464261, 0.34095619,
       0.07259558, 0.14915993])

In [36]:
data_time = data_time.to_numpy()
data_time[o]

array(['2022-06-05', '2022-05-05', '2022-06-21', '2022-04-25',
       '2022-05-06', '2022-06-23', '2022-06-19', '2022-05-11',
       '2022-05-17', '2022-04-19', '2022-05-19', '2022-04-12',
       '2022-04-26', '2022-05-24', '2022-04-14', '2022-05-23',
       '2022-05-12', '2022-04-17', '2022-05-25', '2022-04-13',
       '2022-05-07', '2022-06-16', '2022-05-01', '2022-05-22',
       '2022-04-27', '2022-04-16', '2022-06-06', '2022-06-18',
       '2022-05-27', '2022-06-17', '2022-05-02', '2022-05-04',
       '2022-05-13', '2022-06-22', '2022-05-09', '2022-05-26',
       '2022-04-15', '2022-05-18', '2022-05-08', '2022-04-20',
       '2022-06-20', '2022-04-30'], dtype=object)